In [1]:
import google.generativeai as genai
import json
import re
from tqdm import tqdm # 用於顯示進度條
import csv
import os

# --- 1. 常數與提示詞定義 (來自您的 GRPO 腳本) ---

SYSTEM_PROMPT = """你是一名專門根據患者用藥與病史紀錄預測 ICD-10 編碼的醫學專家。  
請嚴格遵守以下作答格式：  

<reasoning>  
逐步推理：逐條說明病例中每個臨床資訊（症狀、診斷、檢查結果、既往病史、用藥等）可能對應的 ICD-10 編碼，並在最後總結出最合理的編碼集合。  
</reasoning>  
<answer>  
僅輸出最終的 ICD-10 編碼，使用英文逗號分隔，不要添加任何額外文字或解釋。  
</answer>  
"""
EXAMPLE_QUESTION = """
Sex: F
Service: MEDICINE
Allergies: Aspirin
Attending: ___
Chief Complaint: weakness, diarrhea
Major Surgical or Invasive Procedure: None

History of Present Illness:
Ms. ___ is a ___ year-old woman with PMH significant for chronic anemia, osteoporosis, hypertension, ataxia, and recent L5 fracture in the setting of recurrent falls who presents from home with fatigue and generalized weakness and diarrhea.  

The patient's recent history is notable for the follow:  
- On ___, she presented with 4 days of LBP s/p fall from standing at which time imaging revealed acute L5 fracture. She was evaluated by Spine team who recommended early mobilization, pain control, but no brace required. She was evaluated by ___, and discharged to ___.  
- She was discharged home with ___ on ___.  
- On ___, she again presented to ___ s/p fall from standing while trying to reach for a glass of water. She did have a occipital scalp hematoma, but imaging including ___, C-spine CT, and L hip X-ray were negative for acute process so patient was discharged home.  

She now represents with generalized fatigue and diarrhea. In the setting of opiates for her L5 fracture, the patient has had constipation (5 days with no BM) for which she took a "natural laxative" the evening prior to presentation. The patient had 2 bowel movements in the morning of presentation and one episode of incontinence with diarrhea while sleeping. In this setting, she felt very weak and called EMS and was brought to ___ ED.

What ICD-10 codes should be assigned? Please just tell me which codes in the end
"""
EXAMPLE_REASONING = """The patient has chronic anemia, documented and clinically relevant, which corresponds to D500.  
There is vitamin deficiency suggested, which fits E538.  
She also has visual impairment history, which maps to H548.  
Hypertension is a chronic condition, coded as I10.  
Diarrhea due to laxative use is coded as K521 (toxic gastroenteritis and colitis).  
Her recent fracture with underlying osteoporosis corresponds to M810.  
Ataxia is a chronic neurologic condition, coded as R270.  
There is adverse effect of drugs (laxatives/opioids), so T474X5A applies.  
Environmental factor of injury is Y92099 (unspecified place of occurrence).  
She also has aspirin allergy, captured by Z9181.  

Therefore, the most appropriate ICD-10 codes are:
"""

EXAMPLE_ANSWER = "D500 E538 H548 I10 K521 M810 R270 T474X5A Y92099 Z9181"

c:\Users\Yuan\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def extract_icd_answer(text: str) -> str:
    """
    從模型的完整回覆或標準答案中提取 ICD-10 編碼。
    優先從 <answer> 標籤中提取,若無則從整個文本中提取。
    """
    # 使用正則表達式搜尋 <answer> 標籤，re.DOTALL 讓 . 可匹配換行符，re.IGNORECASE 忽略大小寫
    answer_match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL | re.IGNORECASE)
    # 如果找到 <answer> 標籤，則只在標籤內容中搜尋 ICD 編碼；否則在整個文本中搜尋
    search_text = answer_match.group(1) if answer_match else text
    
    # 匹配格式：字母 + 1-3位數字 + 可選的小數點和1-2位數字
    codes_list = re.findall(r'\b([A-Z]\d{1,6}(?:\.\d{1,2})?)\b', search_text, re.IGNORECASE)
    
    # 移除小數點、去重、排序、轉大寫，並過濾掉長度<=2的編碼
    cleaned_codes = sorted(set(
        code.replace('.', '').upper() 
        for code in codes_list 
        if len(code.replace('.', '')) > 2
    ))
    return " ".join(cleaned_codes)



def extract_xml_answer(text: str) -> str:
    """提取 <answer> 標籤內的內容"""
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    answer = answer.strip()
    return answer


def is_text(text: str) -> bool:
    """判斷輸入字串是否主要為文字敘述（而非 ICD code 等短編碼）"""
    letters_and_spaces = sum(c.isalpha() or c.isspace() for c in text)
    ratio = letters_and_spaces / max(1, len(text))  
    return ratio > 0.5


def compute_f1(pred: str, true: str) -> float:
    """
    計算 F1 分數 (前三個字符匹配即算正確)
    
    Args:
        pred: 預測的 ICD 編碼字串,例如 "A010 B021"
        true: 真實的 ICD 編碼字串,例如 "A010 C032"
    
    Returns:
        F1 分數 (0.0 到 1.0)
    """
    # 將字串轉換為 set
    pred = pred.replace('.', '')
    true = true.replace('.', '')
    
    def parse_codes(text):
        if ',' in text:
            return set(code.strip().upper() for code in text.split(',') if code.strip())
        else:
            return set(code.strip().upper() for code in text.split() if code.strip())
    
    pred_set = parse_codes(pred)
    true_set = parse_codes(true)
    
    # 如果都是空集合,視為完全匹配
    if not pred_set and not true_set:
        return 1.0
    
    # 轉換為前三個字符的集合
    def get_prefix_set(codes):
        return set(code[:3] for code in codes if len(code) >= 3)
    
    pred_prefix = get_prefix_set(pred_set)
    true_prefix = get_prefix_set(true_set)
    
    # 計算 TP, FP, FN
    tp = len(pred_prefix & true_prefix)
    fp = len(pred_prefix - true_prefix)
    fn = len(true_prefix - pred_prefix)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    
    if precision + recall == 0:
        return 0.0
    
    f1 = 2 * precision * recall / (precision + recall)
    return f1



def count_xml(text: str) -> float:
    """
    檢查 XML 標籤格式的完整性
    
    Returns:
        分數 (0.0 到 0.5),分數越高表示格式越完整
    """
    score = 0.0
    
    if text.count("<reasoning>\n") == 1:
        score += 0.125
    if text.count("\n</reasoning>\n") == 1:
        score += 0.125
    if text.count("\n<answer>\n") == 1:
        score += 0.125
        # 懲罰 </answer> 後有多餘內容
        extra_content = len(text.split("\n</answer>\n")[-1])
        score -= extra_content * 0.001
    if text.count("\n</answer>") == 1:
        score += 0.125
        # 懲罰 </answer> 後有多餘內容
        extra_content = len(text.split("\n</answer>")[-1]) - 1
        score -= max(0, extra_content) * 0.001
    
    return max(0.0, score)  # ✅ 確保不會是負數

In [ ]:
START_INDEX = 170     # 起始索引(從0開始)
END_INDEX = 340     # 結束索引(不包含此索引)

# --- 4. 載入 Gemini 模型 ---

genai.configure(api_key=" ")

print(f"✅ 正在設定模型: gemini-2.5-flash ...")
print(f"📊 處理區間: 第 {START_INDEX} 至 第 {END_INDEX-1} 筆 (共 {END_INDEX-START_INDEX} 筆)\n")

# generation_config = {
#     "temperature": 0,
#     "top_p": 1,
#     "top_k": 1,
#     "max_output_tokens": 8192,
# }

# 建立模型實例
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=SYSTEM_PROMPT
)

print("✅ 模型設定完畢。")

# --- 5. 載入並處理資料 ---
print("\n📂 正在載入 note_icd_data.jsonl ...")
chat_style_data = []

try:
    with open("note_icd_data.jsonl", "r", encoding="utf-8") as f:
        all_lines = f.readlines()
        total_records = len(all_lines)
        
        print(f"📊 檔案總共有 {total_records} 筆資料")
        
        # 驗證索引範圍
        if START_INDEX < 0 or END_INDEX > total_records:
            print(f"❌ 錯誤：索引範圍超出資料範圍 (0-{total_records})")
            exit()
        
        if START_INDEX >= END_INDEX:
            print(f"❌ 錯誤：起始索引必須小於結束索引")
            exit()
        
        # 只讀取指定區間的資料
        lines = all_lines[START_INDEX:END_INDEX]
        print(f"✅ 已選取第 {START_INDEX} 至 第 {END_INDEX-1} 筆，共 {len(lines)} 筆\n")

        for line in tqdm(lines, desc="[Gemini] 載入並預處理資料"):
            item = json.loads(line)
            ground_truth_answer = extract_icd_answer(item["answer"])
            
            chat_style_data.append({
                "answer": ground_truth_answer,
                "raw_question": item["question"]
            })

except FileNotFoundError:
    print("❌ 錯誤：找不到 'note_icd_data.jsonl' 檔案。")
    exit()

print(f"✅ 資料載入完畢，共 {len(chat_style_data)} 筆。\n")

✅ 正在設定模型: gemini-2.5-flash ...
📊 處理區間: 第 170 至 第 339 筆 (共 170 筆)

✅ 模型設定完畢。

📂 正在載入 note_icd_data.jsonl ...
📊 檔案總共有 3852 筆資料
✅ 已選取第 170 至 第 339 筆，共 170 筆



[Gemini] 載入並預處理資料: 100%|██████████| 170/170 [00:00<00:00, 89791.17it/s]

✅ 資料載入完畢，共 170 筆。



In [4]:
# --- 6. 執行推論並計算指標 ---
all_predictions = []
all_ground_truths = []
all_f1_scores = []
all_completion_lengths = []

# 生成帶區間標記的 CSV 檔名
csv_filename = f"gemini_flash_results_{START_INDEX}_{END_INDEX}.csv"

# 檢查檔案是否已存在
file_exists = os.path.exists(csv_filename)

with open(csv_filename, 'a', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['index', 'accuracy_reward', 'completion_length', 'ground_truths_ans', 'predictions_ans', 'assistant_reply']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # 只在檔案不存在時寫入表頭
    if not file_exists:
        writer.writeheader()
        print(f"📁 CSV 檔案已建立: {csv_filename}")
    else:
        print(f"📁 CSV 檔案已存在，將追加資料: {csv_filename}")
    
    print(f"📊 開始處理區間 {START_INDEX}-{END_INDEX-1}\n")
    
    # 開始推論
    for idx, item in enumerate(tqdm(chat_style_data, desc="[Gemini] 執行推論與評估"), START_INDEX):
        try:
            question = item["raw_question"]
            gt_answer = item["answer"]
            
            # 建立包含 one-shot 範例的完整對話
            chat = model.start_chat(history=[
                {"role": "user", "parts": [EXAMPLE_QUESTION]},
                {"role": "model", "parts": [f"<reasoning>{EXAMPLE_REASONING}</reasoning><answer>{EXAMPLE_ANSWER}</answer>"]}
            ])
            
            # 發送問題並取得回應
            response = chat.send_message(question)
            assistant_reply = response.text
            
            # 印出完整資訊
            print("\n" + "="*80)
            print(f"📝 樣本 #{idx} (絕對索引)")
            print("="*80)
            print(f"\n🤖 模型完整回答:")
            print("-"*80)
            print(assistant_reply)
            print("-"*80)
            
            # 記錄 ground truth
            all_ground_truths.append(gt_answer)
            
            # 計算 completion length
            completion_length = len(assistant_reply)
            print(f"\n📊 完成的回覆長度 (字元數): {completion_length}")
            all_completion_lengths.append(completion_length)
            
            # 提取預測的 ICD 編碼
            predicted_codes = extract_icd_answer(assistant_reply)
            print(f"🔍 提取的 ICD-10 編碼: {predicted_codes}")
            all_predictions.append(predicted_codes)
            
            print(f"\n🎯 Ground Truth Answer:")
            print(f"   {gt_answer}")
            
            # 計算 F1 分數
            f1_score = compute_f1(predicted_codes, gt_answer)
            print(f"✅ F1 分數: {f1_score:.4f}")
            all_f1_scores.append(f1_score)
            
            # 寫入 CSV (包含絕對索引)
            writer.writerow({
                'index': idx,
                'accuracy_reward': f1_score,    
                'completion_length': completion_length,
                'ground_truths_ans': gt_answer if isinstance(gt_answer, str) else json.dumps(gt_answer), 
                'predictions_ans': json.dumps(predicted_codes) if isinstance(predicted_codes, list) else predicted_codes,
                'assistant_reply': assistant_reply if isinstance(assistant_reply, str) else json.dumps(assistant_reply)
            })
            csv_file.flush()
            
            print(f"💾 結果已儲存至 {csv_filename} (第 {idx} 筆)")
            print("="*80 + "\n")
            
        except Exception as e:
            print(f"\n❌ 處理第 {idx} 筆資料時發生錯誤:")
            print(f"   錯誤訊息: {e}")
            print(f"   跳過此筆資料繼續執行...\n")
            continue

print("\n" + "="*80)
print("🎉 推論完成!")
print("="*80)

# --- 7. 顯示最終統計 ---
if not all_predictions:
    print("⚠️  沒有可評估的結果。")
else:
    print("\n" + "="*50)
    print(f"  Gemini-2.5-Flash 區間統計 [{START_INDEX}-{END_INDEX-1}]")
    print("="*50)
    print(f"✅ 總共處理: {len(all_f1_scores)} 筆資料")
    print(f"📊 平均 F1 分數: {sum(all_f1_scores)/len(all_f1_scores):.4f}")
    print(f"📏 平均回覆長度: {sum(all_completion_lengths)/len(all_completion_lengths):.1f} 字元")
    print(f"💾 結果已完整儲存至: {csv_filename}")
    print("="*50)

📁 CSV 檔案已建立: gemini_flash_results_170_340.csv
📊 開始處理區間 170-339



[Gemini] 執行推論與評估:   1%|          | 1/170 [00:20<56:55, 20.21s/it]


📝 樣本 #170 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **Chief Complaint: Worsening foot pain** - This is a symptom, the underlying cause will be coded.
2.  **Major Surgical or Invasive Procedure: Angioplasty (Right AT occlusion), Foot Debridement (ulceration down to and including bone of ___ metatarsal, right foot)** - These procedures confirm severe peripheral vascular disease, ulceration, and bone involvement.
3.  **PMHx for CAD:** 冠狀動脈疾病 (Coronary Artery Disease) - **I25.10** (Atherosclerotic heart disease of native coronary artery without angina pectoris).
4.  **Significant PVD s/p multiple interventions:** 周邊血管疾病 (Peripheral Vascular Disease) 且有多次介入治療史 - **I73.9** (Peripheral vascular disease, unspecified). This is a broad term, but appropriate given the history.
5.  **ESRD on HD:** 末期腎病 (End-Stage Renal Disease) 且需血液透析 (Hemodialysis) - **N18.6** (End-stage renal disease) and **Z99.2** (Dependence on ren

[Gemini] 執行推論與評估:   1%|          | 2/170 [00:23<29:18, 10.46s/it]


📝 樣本 #171 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1. **Chief Complaint: Right thyroid neoplasm.** 這個主訴直接指向甲狀腺的腫瘤。在沒有進一步說明腫瘤是良性、惡性或性質未定的情況下，最合適的編碼是「甲狀腺不確定或未知行為的贅生物」。
2. **Major Surgical or Invasive Procedure: Right thyroid lobectomy.** 雖然進行了甲狀腺葉切除術，這通常是用於治療腫瘤，但此資訊本身不提供腫瘤性質的更多細節，因此仍需依賴主訴的描述。

基於以上資訊，最合理的編碼是描述甲狀腺腫瘤但其性質尚未明確的編碼。
</reasoning>
<answer>
D44.0
</answer>
--------------------------------------------------------------------------------

📊 完成的回覆長度 (字元數): 331
🔍 提取的 ICD-10 編碼: D440

🎯 Ground Truth Answer:
   C73 E039 E785 F1011 F329 F4000 F40240 F419 G47419 G8918 I10 I8510 I951 J45909 K219 K7011 K7290 K7460 R490 Z9884
✅ F1 分數: 0.0000
💾 結果已儲存至 gemini_flash_results_170_340.csv (第 171 筆)



...


📝 樣本 #337 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **Chief Complaint: nausea and pre-syncope**：
    *   Nausea (噁心) 對應 ICD-10 編碼 R11.0。
    *   Pre-syncope (昏厥前狀態) 對應 ICD-10 編碼 R55 (Syncope and collapse)。
2.  **History of Present Illness (HPI) - Symptoms**:
    *   **Diaphoresis (盜汗)**: 對應 ICD-10 編碼 R61 (Hyperhidrosis)。
    *   **Looked pale (面色蒼白)**: 對應 ICD-10 編碼 R23.1 (Pallor)。
    *   **Episode of vomitus (嘔吐發作)**: 對應 ICD-10 編碼 R11.10 (Vomiting, unspecified)。
3.  **History of Present Illness (HPI) - Past Medical History (PMH)**:
    *   **RA with ILD (類風濕性關節炎伴間質性肺病)**:
        *   Rheumatoid arthritis (類風濕性關節炎): 對應 ICD-10 編碼 M05.9 (Rheumatoid arthritis, unspecified)。
        *   Interstitial Lung Disease (ILD) (間質性肺病) 伴隨其他疾病: 對應 ICD-10 編碼 J99.1 (Pulmonary disorders in other diseases classified elsewhere)，該編碼需與M05.9一起使用。
    *   **COPD (慢性阻塞性肺病)**: 對應 ICD-10 編碼 J44.9 (Chronic obstructive pulmonary diseas

[Gemini] 執行推論與評估:  99%|█████████▉| 169/170 [44:07<00:15, 15.86s/it]


📝 樣本 #338 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **Chief Complaint: fall with right arm and pelvic pain**: 這指出了患者的來診原因和主要症狀。
    *   **Fall (跌倒)**：需要一個外部原因碼來描述跌倒事件。由於沒有詳細說明跌倒的具體原因（如絆倒、滑倒），最合適的是使用 W19.XXXA (Unspecified fall, initial encounter) 表示非特指的跌倒，且是初次就診。
    *   **Right arm pain (右臂疼痛)**：這是一個症狀。在沒有具體診斷（如骨折、扭傷）的情況下，可以使用 M79.604 (Pain in right arm) 來編碼。
    *   **Pelvic pain (骨盆疼痛)**：這也是一個症狀。在沒有具體診斷的情況下，可以使用 R10.2 (Pelvic and perineal pain) 來編碼。

2.  **History of Present Illness: fell in her back yard this morning and landed on right hand, right face.**：這提供了跌倒的細節和受傷部位。
    *   **Back yard (後院)**：這是跌倒發生的地點，需要一個地點碼。Y92.013 (Yard of single-family house) 最為貼切。
    *   **Landed on right hand (落在右手)**：這暗示右手有受傷。在沒有明確診斷（如骨折、扭傷）的情況下，S69.91XA (Unspecified injury of right hand, initial encounter) 是合適的。
    *   **Landed on right face (落在右臉)**：這暗示臉部有受傷。S09.90XA (Unspecified injury of head, initial encounter) 可涵蓋臉部

[Gemini] 執行推論與評估: 100%|██████████| 170/170 [44:27<00:00, 15.69s/it]


📝 樣本 #339 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **esophageal adenocarcinoma**: 這是患者的潛在主要診斷，導致了所有後續的治療和併發症。對應的 ICD-10 編碼是 C15.9 (Malignant neoplasm of esophagus, unspecified)。
2.  **EGD with stent removal and replacement**: 這是本次入院的主要目的。患者在之前的手術後放置了食道支架。食道支架的存在是進行此程序的原因。對應的 ICD-10 編碼是 Z95.820 (Presence of esophageal stent)。
3.  **necrotic anastomosis with recreation of esophagogastric anastomosis; buttressing of anastomosis with intercostal muscle; and a leak**: 這表明患者過去有嚴重的食道切除術後併發症，包括吻合口壞死和滲漏，導致需要多次手術和管理。雖然這些是過去的事件，但它們是導致患者複雜情況和需要支架的關鍵，因此是重要的既往病史和消化系統的後遺症。對應的 ICD-10 編碼是 K91.89 (Other postprocedural complications and disorders of digestive system, not elsewhere classified)。
4.  **laparoscopic jejunostomy feeding tube**: 患者放置了空腸造瘻管以提供營養支持，這是一個當前存在的狀態。對應的 ICD-10 編碼是 Z93.4 (Artificial opening of small intestine)。
</reasoning>
<answer>
C15.9,Z95.820,K91.89,Z93.4
</answer>
-------------------------------------

In [5]:
import re

def extract_icd_answer(text: str) -> str:
    """
    從模型的完整回覆或標準答案中提取 ICD-10 編碼。
    優先從 <answer> 標籤中提取,若無則從整個文本中提取。
    """
    # 使用正則表達式搜尋 <answer> 標籤，re.DOTALL 讓 . 可匹配換行符，re.IGNORECASE 忽略大小寫
    answer_match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL | re.IGNORECASE)
    # 如果找到 <answer> 標籤，則只在標籤內容中搜尋 ICD 編碼；否則在整個文本中搜尋
    search_text = answer_match.group(1) if answer_match else text
    
    # 匹配格式：字母 + 1-3位數字 + 可選的小數點和1-2位數字
    codes_list = re.findall(r'\b([A-Z]\d{1,6}(?:\.\d{1,2})?)\b', search_text, re.IGNORECASE)
    
    # 移除小數點、去重、排序、轉大寫，並過濾掉長度<=2的編碼
    cleaned_codes = sorted(set(
        code.replace('.', '').upper() 
        for code in codes_list 
        if len(code.replace('.', '')) > 2
    ))
    return " ".join(cleaned_codes)


# 測試字串
test_text = """D550,O09212,O26872,O30042,O99012,Z3A18。"""

# 執行測試
result = extract_icd_answer(test_text)
print(f"提取結果: {result}")
print(f"預期結果: D550 O09212 O26872 O30042 O99012 Z3A18")
print(f"測試通過: {result == 'R074 R109'}")

提取結果: D550 O09212 O26872 O30042 O99012
預期結果: D550 O09212 O26872 O30042 O99012 Z3A18
測試通過: False
